In [51]:
import numpy as np
import strawberryfields as sf
from csd.codeword import CodeWord

In [54]:
NUM_SHOTS = 1
number_modes = 2
batch_size = 2**number_modes
alpha_val = 0.7
batch = generate_all_codewords(word_size=number_modes, alpha_value=alpha_val)

In [55]:
def single_layer(params):
    # Creates a single mode quantum "program".
    # https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.Program.html
    prog = sf.Program(number_modes)

    # Instantiate the Gaussian backend.
    # https://strawberryfields.readthedocs.io/en/stable/introduction/circuits.html
    eng = sf.Engine("tf", backend_options={"cutoff_dim": 5, "batch_size": batch_size})

    with prog.context as q:
        # Phase space squeezing gate.
        # https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.ops.Sgate.html
        sf.ops.Dgate(params["displacement_magnitude"])  | q[0]
        sf.ops.Dgate(params["displacement_magnitude"])  | q[1]

        # Measures whether a mode contains zero or nonzero photons.
        # https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.ops.MeasureThreshold.html
        sf.ops.MeasureFock()                       | q

    return eng.run(prog, shots=NUM_SHOTS)

In [56]:
# Parameters
params = {
    "displacement_magnitude": 0.5
}

In [57]:
# Execute the single layer of the quantum "program".
result = single_layer(params=params)

In [58]:
# Obtain results.
print(result.samples)

tf.Tensor(
[[[0 0]]

 [[0 0]]

 [[0 1]]

 [[0 0]]], shape=(4, 1, 2), dtype=int32)


In [60]:
[print(sample) for sample in result.samples]

tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)
tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)
tf.Tensor([[0 1]], shape=(1, 2), dtype=int32)
tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)


[None, None, None, None]

In [61]:
one_sample = result.samples[0]

In [70]:
one_sample[0]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([0, 0], dtype=int32)>

In [72]:
convert_batch_sampling_output_to_codeword_list(alpha_value=0.5, batch_samples=result.samples)

tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)
[0.5, 0.5]
tf.Tensor([0.5 0.5], shape=(2,), dtype=float32)
tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)
[0.5, 0.5]
tf.Tensor([0.5 0.5], shape=(2,), dtype=float32)
tf.Tensor([[0 1]], shape=(1, 2), dtype=int32)
[0.5, -0.5]
tf.Tensor([ 0.5 -0.5], shape=(2,), dtype=float32)
tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)
[0.5, 0.5]
tf.Tensor([0.5 0.5], shape=(2,), dtype=float32)


[tf.Tensor([0.5 0.5], shape=(2,), dtype=float32),
 tf.Tensor([0.5 0.5], shape=(2,), dtype=float32),
 tf.Tensor([ 0.5 -0.5], shape=(2,), dtype=float32),
 tf.Tensor([0.5 0.5], shape=(2,), dtype=float32)]

In [66]:
def convert_batch_sampling_output_to_codeword_list(alpha_value: float, batch_samples) -> List[CodeWord]:
    return [convert_sampling_output_to_codeword(alpha_value=alpha_value, one_codeword_output=one_codeword_output) for one_codeword_output in batch_samples]

In [71]:
def convert_sampling_output_to_codeword(alpha_value: float, one_codeword_output) -> CodeWord:
    ON = -1
    OFF = 1
    print(one_codeword_output)
    word = [alpha_value * (ON if one_mode_output != 0 else OFF) for one_mode_output in one_codeword_output[0]]
    print(word)
    tf_word=tf.constant(word)
    cw = CodeWord(word=tf_word)
    print(cw)
    return cw


In [11]:
codeword = convert_sampling_output_to_codeword(alpha_value=0.5, 
                                               tf_samples=result.samples)

tf.Tensor([0 0], shape=(2,), dtype=int64)
[0.5, 0.5]
tf.Tensor([0.5 0.5], shape=(2,), dtype=float32)


In [12]:
print(codeword)

tf.Tensor([0.5 0.5], shape=(2,), dtype=float32)


In [13]:
results = [single_layer(params=params).samples for _ in range(100)]

In [14]:
codewords = [convert_sampling_output_to_codeword(alpha_value=0.5, 
                                               tf_samples=result) for result in results]

tf.Tensor([2 0], shape=(2,), dtype=int64)
[-0.5, 0.5]
tf.Tensor([-0.5  0.5], shape=(2,), dtype=float32)
tf.Tensor([2 0], shape=(2,), dtype=int64)
[-0.5, 0.5]
tf.Tensor([-0.5  0.5], shape=(2,), dtype=float32)
tf.Tensor([1 0], shape=(2,), dtype=int64)
[-0.5, 0.5]
tf.Tensor([-0.5  0.5], shape=(2,), dtype=float32)
tf.Tensor([0 0], shape=(2,), dtype=int64)
[0.5, 0.5]
tf.Tensor([0.5 0.5], shape=(2,), dtype=float32)
tf.Tensor([0 0], shape=(2,), dtype=int64)
[0.5, 0.5]
tf.Tensor([0.5 0.5], shape=(2,), dtype=float32)
tf.Tensor([0 1], shape=(2,), dtype=int64)
[0.5, -0.5]
tf.Tensor([ 0.5 -0.5], shape=(2,), dtype=float32)
tf.Tensor([1 0], shape=(2,), dtype=int64)
[-0.5, 0.5]
tf.Tensor([-0.5  0.5], shape=(2,), dtype=float32)
tf.Tensor([0 0], shape=(2,), dtype=int64)
[0.5, 0.5]
tf.Tensor([0.5 0.5], shape=(2,), dtype=float32)
tf.Tensor([0 0], shape=(2,), dtype=int64)
[0.5, 0.5]
tf.Tensor([0.5 0.5], shape=(2,), dtype=float32)
tf.Tensor([0 1], shape=(2,), dtype=int64)
[0.5, -0.5]
tf.Tensor([ 0.5 -0.5],

In [139]:
one_result = results[0][0]

In [148]:
one_result

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([0, 0])>

In [152]:
tf_alpha = tf.constant(([alpha_value, alpha_value]))

In [154]:
tf.math.multiply(one_result,tf_alpha)

InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a int64 tensor but is a float tensor [Op:Mul]

In [155]:
alpha_value = 0.5
ON = -1
OFF = 1
word = [alpha_value * (ON if sample != 0 else OFF) for sample in one_result]

In [158]:
tf.constant(word)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.5, 0.5], dtype=float32)>

In [52]:
import os
import tensorflow as tf
import random
from typing import List
import itertools
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [53]:
def generate_all_codewords(word_size: int, alpha_value: float) -> List[List[float]]:
    letters = [alpha_value, -alpha_value]
    codewords = [CodeWord(size=len(word), alpha_value=alpha_value, word=list(word))
            for word in itertools.product(letters, repeat=word_size)]
    return [codeword.word for codeword in codewords]

In [50]:

tf_b1 = tf.Variable(0.1)
tf_b2 = tf.Variable(0.1)

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
steps = 40

number_modes = 2
batch_size = 2**number_modes
alpha_val = 0.7
batch = generate_all_codewords(word_size=number_modes, alpha_value=alpha_val)
print(batch)
for step in range(steps):    
    eng = sf.Engine(backend="tf", backend_options={
        "cutoff_dim": 7,
        "batch_size": batch_size,
    })
    
    circuit = sf.Program(number_modes)
    
    b1 = circuit.params("b1")
    b2 = circuit.params("b2")
    alpha1 = circuit.params("alpha1")
    alpha2 = circuit.params("alpha2")

    with circuit.context as q:
        sf.ops.Dgate(alpha1, 0.0) | q[0]
        sf.ops.Dgate(alpha2, 0.0) | q[1]
        sf.ops.Dgate(b1, 0.0) | q[0]
        sf.ops.Dgate(b2, 0.0) | q[1]
        sf.ops.MeasureFock()  | q
        
    with tf.GradientTape() as tape:
        results = eng.run(circuit, args={
            "alpha1": alpha_val,
            "alpha2": alpha_val,
            "b1": tf_b1,
            "b2": tf_b2
        })
        print(f"Result samples: {result.samples}")
        codeword = convert_sampling_output_to_codeword(alpha_value=alpha_val, 
                                               tf_samples=result.samples)
        
        print(f"codeword: {codeword}")
        # get the probability of |0>
        p_zero = results.state.fock_prob([0])
        print(f"p_zero: {p_zero}")
        print(f"bach_size: {batch_size}")
                
        loss /= p_zero/batch_size
        print(f"loss: {loss}")
        
    gradients = tape.gradient(loss, [tf_b1, tf_b2])
    opt.apply_gradients(zip(gradients, [tf_b1,tf_b2]))
    
    if (step + 1) % 10 == 0:
        print("Learned displacement value at step {}: {}".format(step+1, tf_displacement_magnitude.numpy()))

[[0.7, 0.7], [0.7, -0.7], [-0.7, 0.7], [-0.7, -0.7]]
Result samples: [[[0 0]]

 [[0 0]]

 [[1 0]]

 [[0 0]]]
tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()